In [ ]:
import tweepy
import pandas as pd
import re
from textblob import TextBlob
import plotly.express as px
import warnings
warnings.filterwarnings("ignore")

# API Keys

In [ ]:
### Api Keys
counsumer_keys = '<>'
consumer_secret = '<>'
access_token = '<>'
access_secret = '<>'

# Mining

In [ ]:
auth = tweepy.OAuthHandler(counsumer_keys,consumer_secret)
auth.set_access_token(access_token,access_secret)
api = tweepy.API(auth,wait_on_rate_limit=True)

In [ ]:
number_of_tweets = 6000
tweets = []
likes = []

In [ ]:
for i in tweepy.Cursor(api.search,q="ukraine",lang="en",since="2022-02-01",tweet_mode='extended').items(number_of_tweets):
  tweets.append(i.full_text)
  likes.append(i.favorite_count)

In [ ]:
preclean_tweet = pd.DataFrame({'Tweets':tweets,'Likes':likes})
preclean_tweet.head()

,Tweets,Likes
0,@Hepworthclare And he made a point of mentioni...,0
1,RT @johniabraha: Ukrain Embassy in #Ethiopia r...,0
2,RT @johniabraha: Ukrain Embassy in #Ethiopia r...,0
3,@IuliiaMendel make sense. \nno farming in Ukra...,0
4,It's almost an end of April and we are where w...,0


In [ ]:
preclean_tweet.shape

(6000, 2)

In [ ]:
def clean_tweet(text):
  text = text.lower()
  text = re.sub(r'@[A-Za-z0-9]+','',text) ###Removes Mentinons
  text = re.sub(r'[^a-z0-9 ]+','',text) ###Remove special char
  text = re.sub(r'#','',text) ### Remove Hastag
  text = re.sub(r'[[]!"#$%\'()\*+,-./:;<=>?^_`{|}]+','', text) ###Remove Punctuations
  text = re.sub(r'^\s+|\s+$',' ', text) ###Remove additional white sapces
  text = re.sub(r'https?:\/\/\S+','',text) ### Remove http links
  return text

In [ ]:
preclean_tweet['Tweets'] = preclean_tweet['Tweets'].apply(clean_tweet)

In [ ]:
preclean_tweet.head()

,Tweets,Likes
0,and he made a point of mentioning ukrain befo...,0
1,rt ukrain embassy in ethiopia reveals that ru...,0
2,rt ukrain embassy in ethiopia reveals that ru...,0
3,make sense no farming in ukrain actually,0
4,its almost an end of april and we are where we...,0


In [ ]:
###Removing Retweets
preclean_tweet =preclean_tweet[~preclean_tweet.Tweets.str.contains('rt')]
preclean_tweet.shape

(3346, 2)

In [ ]:
preclean_tweet

,Tweets,Likes
3,make sense no farming in ukrain actually,0
4,its almost an end of april and we are where we...,0
5,flying highrussia fighting undeclared world wa...,0
6,if youre an igbo politician or african politic...,1
7,ooook ukrain is burning httpstcoc1dxutzg6k,0
...,...,...
5991,just because it russia where was usa and cnn ...,2
5992,the eu should be ashamed of their handling of ...,0
5996,com in other words us preping taiwan to be a p...,2
5997,at ukrain httpstcop29pbllaxp,0


In [ ]:
def clean_tweet2(text):
  text = re.sub(r'amp',"",text) ### Removing AMP
  return text

In [ ]:
preclean_tweet['Tweets'] = preclean_tweet['Tweets'].apply(clean_tweet2)

In [ ]:
preclean_tweet

,Tweets,Likes
3,make sense no farming in ukrain actually,0
4,its almost an end of april and we are where we...,0
5,flying highrussia fighting undeclared world wa...,0
6,if youre an igbo politician or african politic...,1
7,ooook ukrain is burning httpstcoc1dxutzg6k,0
...,...,...
5991,just because it russia where was usa and cnn ...,2
5992,the eu should be ashamed of their handling of ...,0
5996,com in other words us preping taiwan to be a p...,2
5997,at ukrain httpstcop29pbllaxp,0


In [ ]:
tweets_df = preclean_tweet.reset_index(drop=True)
tweets_df

,Tweets,Likes
0,make sense no farming in ukrain actually,0
1,its almost an end of april and we are where we...,0
2,flying highrussia fighting undeclared world wa...,0
3,if youre an igbo politician or african politic...,1
4,ooook ukrain is burning httpstcoc1dxutzg6k,0
...,...,...
3341,just because it russia where was usa and cnn ...,2
3342,the eu should be ashamed of their handling of ...,0
3343,com in other words us preping taiwan to be a p...,2
3344,at ukrain httpstcop29pbllaxp,0


In [ ]:
tweets_df.insert(0, 'ID', range(1, 1 + len(tweets_df)))

In [ ]:
tweets_df.insert(0, 'User', 'U')

In [ ]:
tweets_df["User_ID"] = tweets_df["User"] + tweets_df["ID"].astype(str)

In [ ]:
tweets_df

,User,ID,Tweets,Likes,User_ID
0,U,1,make sense no farming in ukrain actually,0,U1
1,U,2,its almost an end of april and we are where we...,0,U2
2,U,3,flying highrussia fighting undeclared world wa...,0,U3
3,U,4,if youre an igbo politician or african politic...,1,U4
4,U,5,ooook ukrain is burning httpstcoc1dxutzg6k,0,U5
...,...,...,...,...,...
3341,U,3342,just because it russia where was usa and cnn ...,2,U3342
3342,U,3343,the eu should be ashamed of their handling of ...,0,U3343
3343,U,3344,com in other words us preping taiwan to be a p...,2,U3344
3344,U,3345,at ukrain httpstcop29pbllaxp,0,U3345


In [ ]:
tweets_df.drop(labels=['User','ID'], axis=1,inplace=True)

In [ ]:
tweets_df.head()

,Tweets,Likes,User_ID
0,make sense no farming in ukrain actually,0,U1
1,its almost an end of april and we are where we...,0,U2
2,flying highrussia fighting undeclared world wa...,0,U3
3,if youre an igbo politician or african politic...,1,U4
4,ooook ukrain is burning httpstcoc1dxutzg6k,0,U5


In [ ]:
import nltk
nltk.download('stopwords')


[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
from nltk.corpus import stopwords
stop = stopwords.words('english')
tweets_df['Tweets_wo/stopwords'] = tweets_df['Tweets'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))

# Sentiment

In [ ]:
def getsub(text):
  return TextBlob(text).sentiment.subjectivity

def getpol(text):
  return TextBlob(text).sentiment.polarity

tweets_df['Subjectivity'] = tweets_df['Tweets_wo/stopwords'].apply(getsub)
tweets_df['Polarity'] =   tweets_df['Tweets_wo/stopwords'].apply(getpol)

tweets_df

,Tweets,Likes,User_ID,Tweets_wo/stopwords,Subjectivity,Polarity
0,make sense no farming in ukrain actually,0,U1,make sense farming ukrain actually,0.100000,0.000000
1,its almost an end of april and we are where we...,0,U2,almost end april end august last year lot chan...,0.355556,0.000000
2,flying highrussia fighting undeclared world wa...,0,U3,flying highrussia fighting undeclared world wa...,0.875000,-0.500000
3,if youre an igbo politician or african politic...,1,U4,youre igbo politician african politician alway...,0.122222,-0.222222
4,ooook ukrain is burning httpstcoc1dxutzg6k,0,U5,ooook ukrain burning httpstcoc1dxutzg6k,0.000000,0.000000
...,...,...,...,...,...,...
3341,just because it russia where was usa and cnn ...,2,U3342,russia usa cnn ukrain killed 14000 russian spe...,0.000000,-0.133333
3342,the eu should be ashamed of their handling of ...,0,U3343,eu ashamed handling ukrain crisis concerned ke...,0.333333,-0.166667
3343,com in other words us preping taiwan to be a p...,2,U3344,com words us preping taiwan proxy like ukraine...,0.483333,0.383333
3344,at ukrain httpstcop29pbllaxp,0,U3345,ukrain httpstcop29pbllaxp,0.000000,0.000000


In [ ]:
fig = px.scatter(tweets_df,x = 'Polarity',y='Subjectivity',
           labels={
                     "Polarity": "<--Negative :: Positive--> ",
                     "Subjectivity": "<--Facts :: Opinions--> ",
                 },template='ggplot2',title="General Sentiment")

fig.show()

# Latent Dirichlet Allocation

In [ ]:
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from gensim.models import CoherenceModel
import spacy

In [ ]:
pip install pyLDAvis

     |████████████████████████████████| 1.7 MB 4.3 MB/s 
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
    Preparing wheel metadata ... done
  Created wheel for pyLDAvis: filename=pyLDAvis-3.3.1-py2.py3-none-any.whl size=136898 sha256=3f6405cd73223e22a400d3a8466b41ae1442306f2f801bf41a8281a1851172ec
  Stored in directory: /root/.cache/pip/wheels/c9/21/f6/17bcf2667e8a68532ba2fbf6d5c72fdf4c7f7d9abfa4852d2f
Successfully built pyLDAvis


In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models as gensimvis

/usr/local/lib/python3.7/dist-packages/past/types/oldstr.py:5: DeprecationWarning:

Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated since Python 3.3,and in 3.9 it will stop working



In [ ]:
def sent_to_words(sentences,deacc=True):           ### Deac Removes Punctuations
  for sentence in sentences:
    yield(simple_preprocess(str(sentence)))

### Converting Twwets without stopwords column to a list of list
tweet = tweets_df['Tweets_wo/stopwords']
tweets_data = list(sent_to_words(tweet))


In [ ]:
tweets_data[22]

['lets', 'ask', 'people', 'ukrain', 'wonder', 'give', 'shit', 'mask']

In [ ]:
### Lemmatizzation
pos_tags = ['NOUN','ADV','ADJ','VERB']
nlp = spacy.load('en_core_web_sm',disable=['parser','ner'])

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:126: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.



In [ ]:
def lemmatize(text,allowed_postags = pos_tags):
  tweet_output = []
  for sent in text:
    doc = nlp(" ".join(sent))
    tweet_output.append([token.lemma_ for token in doc if token.pos_ in allowed_postags])
  return tweet_output

In [ ]:
tweets_lemmatize = lemmatize(tweets_data)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecated. Use select.

/usr/local/lib/python3.7/dist-packages/catalogue.py:138: DeprecationWarning:

SelectableGroups dict interface is deprecat

In [ ]:
tweets_lemmatize[22]

['let', 'ask', 'people', 'wonder', 'give', 'shit', 'mask']

Gensim LDA Requires the data in a specific format, LDA requires the corpus to be in the form of a dictionary with words mapped as id whit their frequency.

In [ ]:
id2word = corpora.Dictionary(tweets_lemmatize)

corpus = [id2word.doc2bow(text) for text in tweets_lemmatize]

In [ ]:
[[(id2word[id],freq) for id, freq in cp] for cp in corpus[:1]]

[[('actually', 1), ('farm', 1), ('make', 1), ('sense', 1), ('ukrain', 1)]]

In [ ]:
lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word,
                                            num_topics=10,random_state=100,update_every=1,
                                            chunksize=100,passes=10,alpha=0.1,per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(n

In [ ]:
import pprint
pprint.pprint(lda_model.print_topics())
doc_lda = lda_model[corpus]

[(0,
  '0.094*"help" + 0.033*"leave" + 0.032*"already" + 0.029*"city" + 0.028*"go" '
  '+ 0.026*"ukrain" + 0.024*"know" + 0.024*"bad" + 0.023*"need" + '
  '0.019*"photo"'),
 (1,
  '0.066*"ukrain" + 0.046*"come" + 0.033*"country" + 0.031*"give" + '
  '0.029*"war" + 0.028*"back" + 0.027*"pm" + 0.025*"send" + 0.019*"stop" + '
  '0.019*"money"'),
 (2,
  '0.055*"ukrain" + 0.035*"say" + 0.025*"time" + 0.025*"world" + 0.023*"be" + '
  '0.021*"also" + 0.021*"s" + 0.021*"keep" + 0.020*"force" + 0.019*"go"'),
 (3,
  '0.044*"student" + 0.040*"pakistani" + 0.032*"war" + 0.029*"sink" + '
  '0.020*"civilian" + 0.019*"join" + 0.016*"go" + 0.014*"ship" + '
  '0.012*"threaten" + 0.012*"die"'),
 (4,
  '0.052*"see" + 0.049*"people" + 0.046*"street" + 0.025*"corpse" + '
  '0.022*"body" + 0.021*"could" + 0.019*"news" + 0.019*"lose" + '
  '0.019*"conflict" + 0.015*"show"'),
 (5,
  '0.039*"want" + 0.038*"fight" + 0.038*"ukrain" + 0.031*"ukrainian" + '
  '0.027*"even" + 0.026*"think" + 0.023*"destroy" + 0.022

In [ ]:
coherence = CoherenceModel(model = lda_model,texts = tweets_lemmatize,dictionary = id2word , coherence ='c_v')
coherence_lda = coherence.get_coherence()
print(coherence_lda)

0.37945500711661995


A word has a high lift if it has high relevance in a topic as compared to the overall topic.

The lambda parameter ranks the words in the order of lift.
it is the balance between word frequency and the lift the word got in then topic

In [ ]:
pyLDAvis.enable_notebook()
vis = pyLDAvis.gensim_models.prepare(lda_model,corpus,id2word)
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.038790  0.013576       1        1  12.744926
2     -0.014110  0.168615       2        1  12.193301
5     -0.170031  0.004092       3        1  11.326237
0     -0.066198 -0.063336       4        1  10.638093
6     -0.172503 -0.005431       5        1   9.893326
3      0.236008  0.030407       6        1   8.970201
4     -0.041812 -0.295550       7        1   8.945956
7     -0.027069  0.128829       8        1   8.776971
8      0.228986 -0.076339       9        1   8.315019
9     -0.012061  0.095136      10        1   8.195969, topic_info=        Term        Freq        Total Category  logprob  loglift
67      help  308.000000   308.000000  Default  30.0000  30.0000
215      see  195.000000   195.000000  Default  29.0000  29.0000
59      come  198.000000   198.000000  Default  28.0000  28.0000
98    people  377.000000   377.000000  Default  27.0000  27.0000
1810  street  127.000000   127.000000  Default  26.0000  26.0000
...      ...         ...          ...      ...      ...      ...
313   weapon   74.224501   103.296544  Topic10  -3.5319   2.1710
4     ukrain   98.461781  1053.696735  Topic10  -3.2494   0.1311
139    would   32.030918   171.471802  Topic10  -4.3723   0.8238
123    think   16.374727   152.768860  Topic10  -5.0433   0.2683
668     back   15.570133   158.011159  Topic10  -5.0937   0.1842

[408 rows x 6 columns], token_table=      Topic      Freq      Term
term                           
923       4  0.932724      able
2702      2  0.960068   achieve
0         5  0.980292  actually
381       8  0.967595       add
2662      8  0.976791       age
...     ...       ...       ...
139       4  0.163292     would
139       8  0.647337     would
139      10  0.186620     would
23        8  0.992481      year
75        9  0.988757       yet

[397 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 3, 6, 1, 7, 4, 5, 8, 9, 10])

Hyperparameter tuning

In [ ]:
# compute coherence value at various values of alpha and num_topics
def compute_coherence_values(dictionary, corpus, texts, num_topics_range, alpha_range):

    coherence_values = []
    model_list = []
    for alpha in alpha_range:
        for num_topics in num_topics_range:
            lda_model = gensim.models.ldamodel.LdaModel(corpus=corpus,
                                               id2word=dictionary,
                                               num_topics=num_topics,
                                               alpha=alpha,
                                               per_word_topics=True)
            model_list.append(lda_model)
            coherencemodel = CoherenceModel(model=lda_model, texts=texts, dictionary=dictionary, coherence='c_v')
            coherence_values.append((alpha, num_topics, coherencemodel.get_coherence()))


    return model_list, coherence_values

In [ ]:
# build models across a range of num_topics and alpha
num_topics_range = [2, 6, 10, 15, 20]
alpha_range = [0.01, 0.1, 1]
model_list, coherence_values = compute_coherence_values(dictionary=id2word,
                                                        corpus=corpus,
                                                        texts=tweets_lemmatize,
                                                        num_topics_range=num_topics_range,
                                                        alpha_range=alpha_range)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(n

In [ ]:
coherence_df = pd.DataFrame(coherence_values, columns=['alpha', 'num_topics', 'coherence_value'])
coherence_df

,alpha,num_topics,coherence_value
0,0.01,2,0.283962
1,0.01,6,0.304790
2,0.01,10,0.331998
3,0.01,15,0.330458
4,0.01,20,0.346147
5,0.10,2,0.208928
6,0.10,6,0.298374
7,0.10,10,0.331602
8,0.10,15,0.349897
9,0.10,20,0.344387


In [ ]:
fig2 = px.line(coherence_df, x="num_topics", y="coherence_value", color='alpha',template = 'ggplot2')
fig2.show()

In [ ]:
tune_lda_model = gensim.models.ldamodel.LdaModel(corpus = corpus, id2word = id2word,
                                            num_topics=15,random_state=100,update_every=1,
                                            chunksize=100,passes=10,alpha=0.1,per_word_topics=True)

Streaming output truncated to the last 5000 lines.
/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.

/usr/local/lib/python3.7/dist-packages/gensim/models/ldamodel.py:1077: DeprecationWarning:

Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(n

In [ ]:
import pprint
pprint.pprint(tune_lda_model.print_topics())
doc_lda = tune_lda_model[corpus]

[(0,
  '0.042*"get" + 0.038*"invade" + 0.036*"military" + 0.034*"still" + '
  '0.029*"give" + 0.027*"weapon" + 0.026*"never" + 0.020*"leader" + '
  '0.018*"price" + 0.016*"fact"'),
 (1,
  '0.064*"think" + 0.059*"ukrain" + 0.036*"soldier" + 0.034*"keep" + '
  '0.032*"money" + 0.030*"can" + 0.029*"war" + 0.025*"possible" + 0.021*"life" '
  '+ 0.019*"good"'),
 (2,
  '0.083*"ukrain" + 0.052*"say" + 0.039*"s" + 0.037*"time" + 0.033*"be" + '
  '0.033*"year" + 0.031*"also" + 0.023*"talk" + 0.022*"go" + 0.021*"russian"'),
 (3,
  '0.052*"already" + 0.048*"attack" + 0.047*"city" + 0.043*"sink" + '
  '0.040*"live" + 0.033*"great" + 0.032*"photo" + 0.029*"order" + '
  '0.029*"civilian" + 0.028*"join"'),
 (4,
  '0.050*"even" + 0.042*"shoot" + 0.041*"ask" + 0.033*"corpse" + 0.025*"lose" '
  '+ 0.023*"always" + 0.023*"answer" + 0.023*"inflation" + 0.021*"boy" + '
  '0.020*"look"'),
 (5,
  '0.175*"people" + 0.037*"street" + 0.035*"destroy" + 0.032*"free" + '
  '0.031*"cause" + 0.029*"ukrain" + 0.026*"

In [ ]:
coherence = CoherenceModel(model = tune_lda_model,texts = tweets_lemmatize,dictionary = id2word , coherence ='c_v')
coherence_lda = coherence.get_coherence()
print(coherence_lda)

0.4317980478700891


In [ ]:
pyLDAvis.enable_notebook()
vis2 = pyLDAvis.gensim_models.prepare(tune_lda_model,corpus,id2word)
vis2

PreparedData(topic_coordinates=              x         y  topics  cluster      Freq
topic                                               
6     -0.147276  0.055973       1        1  8.369404
2     -0.178217  0.054345       2        1  8.217658
1     -0.108728  0.125239       3        1  7.378682
5     -0.016557 -0.238576       4        1  7.306665
10    -0.149783  0.006653       5        1  7.244065
4      0.029056 -0.062815       6        1  7.063009
13     0.141880  0.184461       7        1  6.895919
9     -0.109303 -0.069995       8        1  6.737799
0      0.147335  0.016857       9        1  6.660449
8      0.152813  0.152854      10        1  6.531403
12     0.124733  0.009010      11        1  6.240443
3      0.162504 -0.164239      12        1  5.770112
11    -0.128803 -0.013788      13        1  5.666397
14    -0.025043 -0.003399      14        1  5.430011
7      0.105391 -0.052582      15        1  4.487983, topic_info=         Term         Freq        Total Category  logprob  loglift
98     people   397.000000   397.000000  Default  30.0000  30.0000
67       help   271.000000   271.000000  Default  29.0000  29.0000
59       come   210.000000   210.000000  Default  28.0000  28.0000
4      ukrain  1057.000000  1057.000000  Default  27.0000  27.0000
215       see   204.000000   204.000000  Default  26.0000  26.0000
...       ...          ...          ...      ...      ...      ...
5200    adapt     9.130210     9.911396  Topic15  -5.0252   3.0217
5208  johnson     9.121628     9.911647  Topic15  -5.0261   3.0207
5209  lording     9.121628     9.911647  Topic15  -5.0261   3.0207
5204   bottle     9.121626     9.911645  Topic15  -5.0261   3.0207
5205  chequer     9.121626     9.911645  Topic15  -5.0261   3.0207

[544 rows x 6 columns], token_table=      Topic      Freq        Term
term                             
923      14  0.960390        able
496       5  0.956809  absolutely
294       8  0.929126      accept
2702      2  0.962725     achieve
535      12  0.933215      action
...     ...       ...         ...
139       8  0.992218       would
334       7  0.969271       wrong
23        2  0.925892        year
23       10  0.076250        year
75       10  0.985083         yet

[529 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[7, 3, 2, 6, 11, 5, 14, 10, 1, 9, 13, 4, 12, 15, 8])